In [2]:
from pystackreg import StackReg
from skimage import io, exposure
from matplotlib import pyplot as plt

In [3]:
def overlay_images(imgs, equalize=False, aggregator=np.mean):
    if equalize:
        imgs = [exposure.equalize_hist(img) for img in imgs]
    imgs = np.stack(imgs, axis=0)
    
    return aggregator(imgs, axis=0)

In [8]:
import tifffile
import numpy as np

base_path = 'C:/Users/chvre/Desktop/FYP/'
sr = StackReg(StackReg.TRANSLATION)
count = 1

for offset in range(0, 20):
  lt = []

  with tifffile.TiffWriter(base_path + 'image_stack.tif') as stack:
      for i in range(1, 6):
          img_name = base_path + 'AMD_IMAGES/' + str(offset*5 + i) + '.jpg'
          img = io.imread(img_name)
          stack.save(img)

  with tifffile.TiffFile(base_path + 'image_stack.tif') as stack:
    for series in stack.series:
      first_page = series[0]

    lt = stack.asarray(key=slice(None))

  output = sr.register_transform_stack(lt, reference='previous')
  mean = overlay_images(output, aggregator=np.mean)

  noisy_image = io.imread(base_path + 'AMD_IMAGES/' + str(offset*5 + 1) + '.jpg')

  # save images

  tifffile.imwrite(base_path + 'dataset/raw/' + str(count) + '.tif' , noisy_image, photometric='minisblack')
  tifffile.imwrite(base_path + 'dataset/average/' + str(count) + '.tif' , mean, photometric='minisblack')
  count = count + 1

  # # print the noisy and corresponding mean
  # plt.figure()
  # plt.subplot(121)
  # plt.imshow(noisy_image, cmap='gray')

  # plt.subplot(122)
  # plt.imshow(mean, cmap='gray')
  # plt.show()

C:\Users\chvre\AppData\Local\Temp\ipykernel_6068\2911716738.py:16: DeprecationWarning: <tifffile.TiffWriter.save> is deprecated. Use TiffWriter.write
  stack.save(img)
